<a href="https://colab.research.google.com/github/noorwewe/Intelligent-System/blob/master/Glaucoma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
#from tensorflow.keras.applications.efficientnet import EfficientNetB7
#from tensorflow.keras.applications import ResNet101
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential

In [2]:
# Set the paths to your data and label files
data_dir = '/content/drive/MyDrive/Glaucoma/Images_Square'
label_file = '/content/drive/MyDrive/Glaucoma/G1020.csv'

In [3]:
# Set the target size of your input images
target_size = (224, 224)

In [4]:
# Set the batch size for training and validation
batch_size = 32

In [5]:
# Load the label file using pandas
labels_df = pd.read_csv(label_file)

In [6]:
# Create an instance of the ImageDataGenerator class for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [7]:
# Create a training data generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=labels_df,
    directory=data_dir,
    x_col='imageID',
    y_col='binaryLabels',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='raw',
    subset='training'
)

Found 816 validated image filenames.


In [8]:
# Create a validation data generator
val_generator = train_datagen.flow_from_dataframe(
    dataframe=labels_df,
    directory=data_dir,
    x_col='imageID',
    y_col='binaryLabels',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='raw',
    subset='validation'
)

Found 204 validated image filenames.


In [9]:
# Load the pre-trained VGG19 model and exclude the top layer
base_model = VGG19(weights='imagenet', include_top=False, input_shape=target_size+(3,))
#base_model = EfficientNetB7(weights='imagenet', include_top=False, input_shape=target_size+(3,))
#base_model = RestNet101(weights='imagenet', include_top=False, input_shape=target_size+(3,))


80134624/80134624 [==============================] - 1s 0us/step


In [63]:
# Replace the output layer with a single-unit dense layer with sigmoid activation for binary classification
#base_model.layers.pop()
#base_model.layers.pop()
#base_model.outputs = [base_model.layers[-1].output]
#base_model.layers[-1].outbound_nodes = []

#base_model.add(Dense(1, activation='sigmoid'))

In [11]:
# Freeze the weights of the pre-trained layers
#for layer in base_model.layers:
#    layer.trainable = True;


In [10]:
# Add new trainable layers on top of the pre-trained model
model = Sequential([
    base_model,
    Flatten(),
    Dense(4096, activation='relu'),
    Dropout(0),
    Dense(4096, activation='relu'),
    Dropout(0),
    Dense(1, activation='sigmoid')

])

In [11]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 4096)              102764544 
                                                                 
 dropout (Dropout)           (None, 4096)              0         
                                                                 
 dense_1 (Dense)             (None, 4096)              16781312  
                                                                 
 dropout_1 (Dropout)         (None, 4096)              0         
                                                                 
 dense_2 (Dense)             (None, 1)                 4

In [12]:
# Compile the model with an optimizer, loss function, and evaluation metric
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])


In [13]:
# Train the model on the training data and evaluate on the validation data
history = model.fit(train_generator, epochs=3, validation_data=val_generator)

Epoch 1/3
26/26 [==============================] - 2641s 101s/step - loss: 0.8470 - accuracy: 0.6324 - val_loss: 0.5438 - val_accuracy: 0.8333
Epoch 2/3
26/26 [==============================] - 2614s 101s/step - loss: 0.6455 - accuracy: 0.6642 - val_loss: 0.4622 - val_accuracy: 0.8333
Epoch 3/3
26/26 [==============================] - 2616s 101s/step - loss: 0.6341 - accuracy: 0.6789 - val_loss: 0.5119 - val_accuracy: 0.8333


In [14]:
model.save('percobaanVGG19Glaucoma.h5')

In [17]:
from keras.models import load_model
model=load_model('percobaanVGG19Glaucoma.h5')

In [15]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os

# Define the directory containing the images to classify
image_dir = '/content/drive/MyDrive/Glaucoma/Images_Classify'

In [16]:
# Loop through all the images in the directory
for filename in os.listdir(image_dir):
    if filename.endswith('.jpg'):
        # Load the image
        image_path = os.path.join(image_dir, filename)
        image = load_img(image_path, target_size=(224, 224))
        image_array = img_to_array(image)
        image_array = np.expand_dims(image_array, axis=0)

        # Preprocess the image using the preprocess_input function from the VGG19 module
        preprocessed_image = preprocess_input(image_array)

        # Use the model to predict the class of the image
        prediction = model.predict(preprocessed_image)

        # Print the predicted class label
        if prediction > 0.5:
            print(f'{filename} is classified as class 1')
        else:
            print(f'{filename} is classified as class 0')



1/1 [==============================] - 2s 2s/step
image_3184.jpg is classified as class 0
